In [1]:
conda env list

/home/suramya/miniconda3/lib/python3.12/site-packages/conda/base/context.py:201: FutureWarning: Adding 'defaults' to channel list implicitly is deprecated and will be removed in 25.3. 

To remove this warning, please choose a default channel explicitly with conda's regular configuration system, e.g. by adding 'defaults' to the list of channels:

  conda config --add channels defaults

For more information see https://docs.conda.io/projects/conda/en/stable/user-guide/configuration/use-condarc.html

  deprecated.topic(

# conda environments:
#
base                   /home/suramya/miniconda3
cc3d                   /home/suramya/miniconda3/envs/cc3d
cc3d_4.7.0             /home/suramya/miniconda3/envs/cc3d_4.7.0
ebay_ml              * /home/suramya/miniconda3/envs/ebay_ml
llm                    /home/suramya/miniconda3/envs/llm
llm_2                  /home/suramya/miniconda3/envs/llm_2
med_diag_env           /home/suramya/miniconda3/envs/med_diag_env
medical_bot            /home/suramya/mi

In [2]:
import pandas as pd

# Listing titles (2 million rows)
df_listing = pd.read_csv(
    "Listing_Titles.tsv",
    sep="\t",
    keep_default_na=False,  # don't turn "" into NaN
    na_values=None,
    dtype=str
)

# Tagged train data (≈ 56k rows)
df_train = pd.read_csv(
    "Tagged_Titles_Train.tsv",
    sep="\t",
    keep_default_na=False,
    na_values=None,
    dtype=str
)
df_listing = df_listing.rename(columns={
    "Record Number": "record_id",
    "Category": "category_id",
    "Title": "title"
})

df_train = df_train.rename(columns={
    "Record Number": "record_id",
    "Category": "category_id",
    "Title": "title",
    "Token": "token",
    "Tag": "raw_tag"
})
print(df_listing.shape, df_listing.columns)
print(df_train.shape, df_train.columns)


(2000000, 3) Index(['record_id', 'category_id', 'title'], dtype='object')
(56812, 5) Index(['record_id', 'category_id', 'title', 'token', 'raw_tag'], dtype='object')


In [3]:
def record_to_bio(group):
    tokens = group['token'].tolist()
    raw_tags = group['raw_tag'].tolist()
    bios = []

    prev_tag_nonempty = None  # last seen non-empty, non-"O" tag
    prev_raw_tag = None       # raw tag of previous token (can be same string, or "")

    for i, rt in enumerate(raw_tags):
        if rt == "O":
            bios.append("O")
            prev_tag_nonempty = None
            prev_raw_tag = "O"
        elif rt == "":
            # continuation of previous non-empty tag
            if prev_tag_nonempty is None:
                # edge case: blank but we have nothing to continue -> call it "O"
                bios.append("O")
            else:
                bios.append("I-" + prev_tag_nonempty)
            prev_raw_tag = ""
        else:
            # new explicit tag
            # decide if B- or new B- (always B, because spec says even same tag twice can be two entities)
            bios.append("B-" + rt)
            if rt != "O":
                prev_tag_nonempty = rt
            else:
                prev_tag_nonempty = None
            prev_raw_tag = rt

    return {
        "tokens": tokens,
        "bio_tags": bios,
        "category_id": group['category_id'].iloc[0],
        "record_id": group['record_id'].iloc[0],
        "title": group['title'].iloc[0],
    }

grouped = df_train.groupby('record_id', sort=True).apply(record_to_bio)
train_sequences = list(grouped)


/tmp/ipykernel_43550/1208407759.py:40: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df_train.groupby('record_id', sort=True).apply(record_to_bio)


In [4]:
all_labels = sorted({tag for seq in train_sequences for tag in seq["bio_tags"]})
# Example: ["B-Hersteller","B-Kompatibles_Fahrzeug_Modell",...,"I-...","O"]

label2id = {lbl:i for i,lbl in enumerate(all_labels)}
id2label = {i:lbl for lbl,i in label2id.items()}


In [5]:
from transformers import AutoTokenizer

model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def encode_sequence(tokens, bio_tags, category_id):
    # prepend synthetic category token
    cat_token = f"[CAT_{category_id}]"
    input_tokens = [cat_token] + tokens
    input_tags   = ["O"] + bio_tags  # CAT token gets "O"

    enc = tokenizer(
        input_tokens,
        is_split_into_words=True,
        return_offsets_mapping=False,
        truncation=True,
        max_length=128,   # adjust if titles can be longer; most won't
        return_tensors=None,
    )

    # enc.word_ids() gives index of which original input_tokens word each subword came from
    word_ids = enc.word_ids()

    labels = []
    for wi in word_ids:
        if wi is None:
            labels.append(-100)  # special tokens like <s>, </s>
        else:
            # only label the first subword of each original token
            # rule: if this word_id is same as previous word_id, use -100
            if len(labels)>0 and word_ids[labels.index(labels[-1]) if labels[-1]!=-100 else len(labels)-1] == wi:
                # Actually easier: track prev_wi
                pass

def encode_sequence(tokens, bio_tags, category_id):
    cat_token = f"[CAT_{category_id}]"
    input_tokens = [cat_token] + tokens
    input_tags   = ["O"] + bio_tags

    enc = tokenizer(
        input_tokens,
        is_split_into_words=True,
        return_offsets_mapping=False,
        truncation=True,
        max_length=128,
    )

    word_ids = enc.word_ids()

    labels = []
    prev_wi = None
    for wi in word_ids:
        if wi is None:
            labels.append(-100)
        else:
            if wi != prev_wi:
                # first subword of this original token → real label id
                lbl = input_tags[wi]
                labels.append(label2id[lbl])
                prev_wi = wi
            else:
                # continuation subword → ignore in loss
                labels.append(-100)

    return {
        "input_ids": enc["input_ids"],
        "attention_mask": enc["attention_mask"],
        "labels": labels,
    }


/home/suramya/miniconda3/envs/ebay_ml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
encoded_train = [encode_sequence(seq["tokens"], seq["bio_tags"], seq["category_id"])
                 for seq in train_sequences]


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader

class NERDataset(Dataset):
    def __init__(self, enc_list):
        self.data = enc_list
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "input_ids": torch.tensor(item["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(item["attention_mask"], dtype=torch.long),
            "labels": torch.tensor(item["labels"], dtype=torch.long),
        }

train_dataset = NERDataset(encoded_train)


In [9]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)


In [10]:
from transformers import AutoModelForTokenClassification

num_labels = len(label2id)

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./ebay_ner_model",
    per_device_train_batch_size=16,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="epoch",
    # evaluation_strategy="no",  # baseline: no val split yet
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=None,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()


/tmp/ipykernel_43550/1744690733.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/suramya/miniconda3/envs/ebay_ml/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
